In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.regression import LabeledPoint
from sklearn import metrics
from numpy import array
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from pyspark.mllib.classification import SVMWithSGD, SVMModel
sc = SparkContext(appName="ClassificationOfWineDataset")

In [2]:
data = sc.textFile("D://NEU - Big Data and Intelligent Analytics/Assignment 1/winequality-white-classification-parsed format.csv")

In [3]:
def line_to_field(line):
    #space_separated_line = line.replace(';', ' ')
    string_array = line.split(',')
    #float_array = map(float, string_array)
    #float_array[11] = np.where(float_array[10] > 5.0, 1, 0)
    #float_array[11]=(float_array[10] > 5) 
    return LabeledPoint(string_array[0], string_array[1].split(' '))


In [4]:
parsedData = data.map(line_to_field)

In [5]:
#Spliting data into training and test data
(training, test) = parsedData.randomSplit([0.7, 0.3])

In [174]:
# Build the model and passing the parameter regParam as L1 for L1 regularisation
model = SVMWithSGD.train(training, iterations=200,step=1.5,regType="l1", regParam=0.1)

In [175]:
# Evaluating the model on training data
labelsAndPreds = training.map(lambda p: (p.label, model.predict(p.features))).cache()
accuracy_training = labelsAndPreds.filter(lambda (v, p): v == p).count() / float(training.count())
print("Accuracy(training) = " + str(accuracy_training))
x = np.array(labelsAndPreds.collect())

Accuracy(training) = 0.762127410871


In [176]:
# Evaluating the model on test data
labelsAndPreds1 = test.map(lambda p: (p.label, model.predict(p.features))).cache()
accuracy_test = labelsAndPreds1.filter(lambda (v, p): v == p).count() / float(test.count())
print("Accuracy(test) = " + str(accuracy_test))
y = np.array(labelsAndPreds1.collect())

Accuracy(test) = 0.774390243902


In [177]:
#clearing the threshold for getting scores as prediction.
model.clearThreshold()

In [178]:
#making an array of true lables and its predicted scores for training data
scoreandlabels = np.array(training.map(lambda p: (p.label, model.predict(p.features))).collect())
print "Training Area under curve  = " + str(roc_auc_score(scoreandlabels[:,0],scoreandlabels[:,1]))
print "Training Area under Precision-Recall = " + str(average_precision_score(scoreandlabels[:,0],scoreandlabels[:,1]))

Training Area under curve  = 0.659871378857
Training Area under Precision-Recall = 0.309115598221


In [179]:
#making an array of true lables and its predicted scores for test data
scoreandlabels1 = np.array(test.map(lambda p: (p.label, model.predict(p.features))).collect())
print "Test Area under curve  = " + str(roc_auc_score(scoreandlabels1[:,0],scoreandlabels1[:,1]))
print "Test Area under Precision-Recall = " + str(average_precision_score(scoreandlabels1[:,0],scoreandlabels1[:,1]))

Test Area under curve  = 0.668677174656
Test Area under Precision-Recall = 0.327067346282


In [180]:
#Making confusion matrix for training data
confusion_matrix(x[:,0],x[:,1])

array([[2548,  131],
       [ 683,   60]])

In [182]:
#Making confusion matrix for training data
confusion_matrix(y[:,0],y[:,1])

array([[1117,   42],
       [ 291,   26]])

In [183]:
#checking the weights for the various features
model.weights

DenseVector([0.5981, 0.0, 0.0, -1.148, 0.0, 6.3704, -3.8294, 0.0122, 0.1083, 0.0009, 10.9274])

In [220]:
sc.stop()